# Using NumPy with Rasters

In addition to converting feature classes in to NumPy arrays, we can also convert entire raster datasets into 2-dimensional arrays. This allows us, as we'll see below, to programmatically extract values from these rasters, or we can integrate these arrays with other packages to perform custom analyses with the data. 

For more information on this topic see:
https://4326.us/esri/scipy/devsummit-2016-scipy-arcgis-presentation-handout.pdf

In [ ]:
# Import the modules
import arcpy
import numpy as np

In [ ]:
#Import a plotting library and enable in-line viewing of plots
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#Set the name of the file we'll import
demFilename = '../Data/DEM.tif'

In [ ]:
#Import the DEM as a NumPy array, using only a 200 x 200 pixel subset of it
arrDEM = arcpy.RasterToNumPyArray(demFilename)

#### Exploring our DEM array

In [ ]:
#View the raw elevation values
arrDEM

In [ ]:
#What is the shape of the raster (i.e. the # of rows and columns)? 
arrDEM.shape
#...note it's a 2d array

In [ ]:
#Compute stats across the entire DEM
arrDEM.min(), arrDEM.max(),arrDEM.mean()

#### View the values as an image, using the terrain colormap
 * See https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
#Create the canvas, setting the size to 12 x 8 units
plt.figure(figsize=(12,8))
#Add the DEM on the canvas, using the terrain colormap 
plt.imshow(arrDEM,cmap='terrain')
#Display the figure in the notebook
plt.show()

### Subsetting the DEM
We can subset by location (image coordinates, or rows and columns) or by value, using binary masks.

* To subset by image coordinates, we can "slice" the data. Here we'll look at just the  northern most 200 rows and columns 600 to 800.

In [ ]:
#Extract a subset of the DEM
arrDEMsubset = arrDEM[:200,600:800]

In [ ]:
#Display the subset
plt.imshow(arrDEMsubset,cmap='terrain')
plt.show()

In [ ]:
#How about a contour plot!
arrN50 = arrDEM[:200,600:800]
plt.contour(arrDEMsubset,cmap='terrain')
plt.show()

* Or, we can subset by values, i.e. by elevation. Here we'll create a mask of all pixels above 50,000'. The mask generated is the same size as the arrDEM array, but with values of just True (if above 50,000) or False (if not). 

In [ ]:
#Create a mask of pixels with elevation < 50000
arrLT50k_mask = arrDEM < 50000
arrLT50k_mask

* Since true = 1 and false = 0, to get a count of the pixels meeting our criteria, we just compute the sum. 

In [ ]:
#Count the # of pixels above 50000'
arrLT50k_mask.sum()

* And we can map these values

In [ ]:
#Show the values
plt.imshow(arrLT50k_mask);

* Here is a similar analysis all in one line,revealing pixels between 40k' and 60k'.

In [ ]:
#All in one: Show pixels between 40000' and 60000'
plt.imshow((arrDEM > 40000) & (arrDEM < 60000));

## Stats with numpy
We can compute histograms and other goodies with data in a numpy array.

* First, we'll flatten the 2D array into a 1D series of values

In [ ]:
#"Flatten" into a 1D array
arrFlat = arrDEM.flatten()
arrFlat.shape

* NumPy's `histogram` function computes histogram data from our elevations, generating two separate arrays: the first are the cell counts in a bin, and the second lists the elevation values comprising the bin edges. 

In [ ]:
histData,bin_edges = np.histogram(arrFlat,bins=20)
bin_edges

* Or we can plot the values as a histogram.

In [ ]:
plt.hist(arrFlat,bins='auto')
plt.title("Histogram of elevation")
plt.show()

## More advanced analysis

The SciPy package has a number of multi-dimensional image processing capabilities (see https://docs.scipy.org/doc/scipy/reference/ndimage.html). Here is a somewhat complex example that runs through 10 iterations of computing a neighborhood mean (using the nd.median_filter) with an incrementally growing neighorhood. We then subtract that neighborhood median elevation from the original elevation to compute Topographic Position Index (TPI, see http://www.jennessent.com/downloads/tpi-poster-tnc_18x22.pdf)

If you don't fully understand how it works, at least appreciate that converting a raster to a NumPy array enables us to use other packages to execute custom analyses on the data. 

In [ ]:
#Import the SciPy and plotting packages
import scipy.ndimage as nd
from matplotlib import pyplot as plt

#Allows plots in our Jupyter Notebook
%matplotlib inline

#Create a 'canvas' onto which we can add plots
fig = plt.figure(figsize=(20,20))

#Loop through 10 iterations
for i in range(10):
    #Create a kernel, intially 3 x 3, then expanding 3 x 3 at each iteration 
    size = (i+1) * 3
    print (size,end=' ')
    #Compute the median value for the kernel surrounding each pixel
    med = nd.median_filter(arrDEMsubset, size)
    #Subtract the median elevation from the original to compute TPI
    tpi = arrDEMsubset - med
    #Create a subplot frame
    a = fig.add_subplot(5, 5,i+1)
    #Show the median interpolated DEM
    plt.imshow(tpi, interpolation='nearest')
    #Set titles for the plot
    a.set_title('{}x{}'.format(size, size))
    plt.axis('off')
    plt.subplots_adjust(hspace = 0.1)
    prev = med

### Recap
By converting a raster dataset to a NumPy array, we enable a wide array of different analyses on our data. 